# Backups

dbatools is **awesome** at performing backups :-)

The next block sets the variables for tje instances and folder paths for this Notebook and checks the connection - Refer to the first notebook for any issues

In [1]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbatoolsdemo'
$SqlInstances = 'localhost,15592', 'localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Write-Output " Creating connection to the containers"
try {
    $SQL1 = Connect-DbaInstance -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential 
    $SQL2 = Connect-DbaInstance -SqlInstance $SqlInstances[1] -SqlCredential $SqlCredential
    Write-Output "We have a connection to the containers"

}
catch {
    Write-Output "You haven't got a connection to the containers - Either they are still upgrading in which case try again in 30 seconds or the containers have not come up correctly"
    Write-Output "Make sure the containers are running - the code is below in a block for you"
    Write-Output "docker ps -a"
    Write-Output "If they are read the logs - the code is below in a block for you"
    Write-Output "docker logs dbatools_SQL2019_1"
    Write-Output "docker logs dbatools_SQL2019-1_1"
}

Creating connection to the containers

We have a connection to the containers

## Investigate a directory using the SQL Service Account

We can investigate the file system from the viewpoint of the SQL Service Account using `Get-DbaFile` We are going to use that to show the files in the directory but this can be really useful when investigating errors with file system access, for example when backups fail or when reading from shares. This command will try ot access the path from teh SQL Instance using the SQL Account which means that you can troubleshoot the error and also that users do not need access to directories holding backups or client files for example  
If we look in the Directory that we created in the create container notebook using the command below

In [2]:
Get-DbaFile -SqlInstance $SQL1 -SqlCredential $SqlCredential -Path /var/opt/mssql/backups/SQL1

Filename SqlInstance

-------- -----------

/var/opt/mssql/backups/SQL1\dummyfile.txt 9a744bd5ae64

You can see that all there is in the directory is a dummy file that we created earlier when we set up the directories  

## Check the last time that the databases were backed up

Knowing the last time a database was backed up is often useful information and with dbatools it can be retrieved easily. (NB - I use Warning action silently continue to hide the backups from multiple forks warnings that I have created when I created the container and havent fixed yet!)

In [3]:
Get-DbaLastBackup -SqlInstance $SQL1,$SQL2 -SqlCredential $SqlCredential -WarningAction SilentlyContinue | Format-Table 

ComputerName InstanceName SqlInstance Database LastFullBackup LastDiffBackup LastLogBackup

------------ ------------ ----------- -------- -------------- -------------- -------------

localhost MSSQLSERVER 9a744bd5ae64 AdventureWorks2017 2019-12-21 14:42:01.000 2019-12-21 14:42:00.000

localhost MSSQLSERVER 9a744bd5ae64 master 2019-12-21 14:41:56.000

localhost MSSQLSERVER 9a744bd5ae64 model 2019-12-21 14:41:56.000

...

localhost MSSQLSERVER a102342268b9 model 2019-12-21 14:41:56.000

localhost MSSQLSERVER a102342268b9 msdb 2019-12-21 14:41:57.000

localhost MSSQLSERVER a102342268b9 Northwind 2019-12-21 14:42:01.000 2019-12-21 14:42:00.000

localhost MSSQLSERVER a102342268b9 pubs 2019-12-21 14:42:02.000 2019-12-21 14:42:00.000

localhost MSSQLSERVER a102342268b9 WideWorldImporters 2019-12-21 14:42:03.000 2019-12-21 14:42:02.000

The databases were last backed up on the 21st December 2019. Thats not so clever!

## Docker requires permissions

You will get errors for the command below if the account that you have used to allow Docker to access your drives does not have permissions to the folder that we have created. [You can see the instructions here](https://docs.docker.com/docker-for-windows/)

You should add the docker account with full control over your dbatoolsdemo directory in your User Profile directory if you get errors below.

![DockerCompose](.\images\dbatoolsdemopermissions.png )

Its annoying but you can give the docker account modify permissions to your documents folder and it will inherited

![DockerCompose](.\images\documentsdirectorypermissions.png )



## Perform a backup of the entire instance

With one line of code we can quickly backup an entire instance.

In [4]:
Backup-DbaDatabase  -SqlInstance $SQL1 -SqlCredential $SqlCredential -Path /var/opt/mssql/backups/SQL1

PROGRESS: Complete

PROGRESS: Complete

PROGRESS: Complete

PROGRESS: Complete

PROGRESS: Complete

...

9a744bd5ae64 model Full 2.59 MB Disk 2020-02-29 16:48:29.000 00:00:01 2020-02-29 16:48:30.000

9a744bd5ae64 msdb Full 16.09 MB Disk 2020-02-29 16:48:32.000 00:00:02 2020-02-29 16:48:34.000

9a744bd5ae64 Northwind Full 6.71 MB Disk 2020-02-29 16:48:46.000 00:00:04 2020-02-29 16:48:50.000

9a744bd5ae64 pubs Full 4.46 MB Disk 2020-02-29 16:49:06.000 00:00:04 2020-02-29 16:49:10.000

9a744bd5ae64 WideWorldImporters Full 444.85 MB Disk 2020-02-29 16:49:29.000 00:01:18 2020-02-29 16:50:47.000

All of the databases are backed up - Lets have a look in the directory

In [5]:
Get-DbaFile -SqlInstance $SQL1 -SqlCredential $SqlCredential -Path /var/opt/mssql/backups/SQL1

Filename SqlInstance

-------- -----------

/var/opt/mssql/backups/SQL1\AdventureWorks2017_202002291647.bak 9a744bd5ae64

/var/opt/mssql/backups/SQL1\dummyfile.txt 9a744bd5ae64

/var/opt/mssql/backups/SQL1\master_202002291648.bak 9a744bd5ae64

/var/opt/mssql/backups/SQL1\model_202002291648.bak 9a744bd5ae64

/var/opt/mssql/backups/SQL1\msdb_202002291648.bak 9a744bd5ae64

/var/opt/mssql/backups/SQL1\Northwind_202002291648.bak 9a744bd5ae64

/var/opt/mssql/backups/SQL1\pubs_202002291648.bak 9a744bd5ae64

/var/opt/mssql/backups/SQL1\WideWorldImporters_202002291649.bak 9a744bd5ae64

If you want to have a look in the mounted volume on your laptop 

In [6]:
Get-ChildItem "$FolderPath\SQL1"

Directory: C:\Users\mrrob\Documents\dbatoolsdemo\SQL1

Mode LastWriteTime Length Name

---- ------------- ------ ----

-a--- 29/02/2020 16:48 217178112 AdventureWorks2017_202002291647.bak

-a--- 29/02/2020 16:36 0 dummyfile.txt

...

-a--- 29/02/2020 16:48 2744320 model_202002291648.bak

-a--- 29/02/2020 16:48 16896000 msdb_202002291648.bak

-a--- 29/02/2020 16:48 7065600 Northwind_202002291648.bak

-a--- 29/02/2020 16:49 4706304 pubs_202002291648.bak

-a--- 29/02/2020 16:50 466493440 WideWorldImporters_202002291649.bak

## Backup to a directory for each database

Rob - I hear you cry, I have 2000 databases, I dont want them all backing up to the same directory - Can dbatools help me there ?

Sure, just add the `-CreateFolder` switch.

Lets demonstrate with SQL2

Theres nothing up my sleeves, lets look at the SQL2 folder


In [7]:
Get-DbaFile -SqlInstance $SQL2 -SqlCredential $SqlCredential -Path /var/opt/mssql/backups/SQL2

Filename SqlInstance

-------- -----------

/var/opt/mssql/backups/SQL2\dummyfile.txt a102342268b9

Only the dummy file! 

### Backup the databases

In [9]:
Backup-DbaDatabase -SqlInstance $SQL2 -SqlCredential $SqlCredential -CreateFolder -Path /var/opt/mssql/backups/SQL2 -CompressBackup -WarningAction SilentlyContinue

PROGRESS: Complete

PROGRESS: Complete

PROGRESS: Complete

PROGRESS: Complete

PROGRESS: Complete

...

a102342268b9 model Full 2.59 MB Disk 2020-02-29 16:54:23.000 00:00:02 2020-02-29 16:54:25.000

a102342268b9 msdb Full 16.09 MB Disk 2020-02-29 16:54:30.000 00:00:02 2020-02-29 16:54:32.000

a102342268b9 Northwind Full 6.71 MB Disk 2020-02-29 16:54:37.000 00:00:02 2020-02-29 16:54:39.000

a102342268b9 pubs Full 4.46 MB Disk 2020-02-29 16:54:43.000 00:00:02 2020-02-29 16:54:45.000

a102342268b9 WideWorldImporters Full 444.79 MB Disk 2020-02-29 16:54:50.000 00:00:33 2020-02-29 16:55:23.000

## have a look in the directory

In [10]:
Get-ChildItem "$FolderPath\SQL2" -Recurse

Directory: C:\Users\mrrob\Documents\dbatoolsdemo\SQL2

Mode LastWriteTime Length Name

---- ------------- ------ ----

d---- 29/02/2020 16:53 AdventureWorks2017

d---- 29/02/2020 16:54 master

...

Directory: C:\Users\mrrob\Documents\dbatoolsdemo\SQL2\WideWorldImporters

Mode LastWriteTime Length Name

---- ------------- ------ ----

-a--- 29/02/2020 16:53 126488576 WideWorldImporters_202002291653.bak

-a--- 29/02/2020 16:55 126492672 WideWorldImporters_202002291654.bak

As you can see we have the databases backed up, each in their own directory :-)

## Double check all the instances

dbatoos works with multiple instances whereever there is a `-SqlInstance` parameter

Lets check the last time that these databases were backed up on both of the containers

In [10]:
Get-DbaLastBackup -SqlInstance $SQL1,$SQL2 -SqlCredential $SqlCredential -WarningAction SilentlyContinue | Format-Table 

ComputerName InstanceName SqlInstance Database LastFullBackup LastDiffBackup LastLogBackup

------------ ------------ ----------- -------- -------------- -------------- -------------

localhost MSSQLSERVER fe7be92417e3 AdventureWorks2017 2020-02-29 14:48:23.000 2019-12-21 14:42:00.000

localhost MSSQLSERVER fe7be92417e3 master 2020-02-29 14:48:24.000

localhost MSSQLSERVER fe7be92417e3 model 2020-02-29 14:48:25.000

...

localhost MSSQLSERVER 321a65188e44 model 2020-02-29 14:48:58.000

localhost MSSQLSERVER 321a65188e44 msdb 2020-02-29 14:48:58.000

localhost MSSQLSERVER 321a65188e44 Northwind 2020-02-29 14:48:59.000 2019-12-21 14:42:00.000

localhost MSSQLSERVER 321a65188e44 pubs 2020-02-29 14:49:00.000 2019-12-21 14:42:00.000

localhost MSSQLSERVER 321a65188e44 WideWorldImporters 2020-02-29 14:49:04.000 2019-12-21 14:42:02.000

# Don't forget to clean up

Now you can use these containers to run the rest of the notebooks - Don't forget to run the Clean Up Containers Notebook at the end to clean up